# Performing Testing at Each Round of Training 

Use for developing (autoreloads changes made across packages)

In [ ]:
%load_ext autoreload
%autoreload 2

## Start the network
Before running this notebook, start the network with `./scripts/fedbiomed_run network`

## Setting the node up
It is necessary to previously configure a node:
1. `./scripts/fedbiomed_run node add`
  * Select option 2 (default) to add MNIST to the node
  * Confirm default tags by hitting "y" and ENTER
  * Pick the folder where MNIST is downloaded (this is due torch issue https://github.com/pytorch/vision/issues/3549)
  * Data must have been added (if you get a warning saying that data must be unique is because it's been already added)
  
2. Check that your data has been added by executing `./scripts/fedbiomed_run node list`
3. Run the node using `./scripts/fedbiomed_run node run`. Wait until you get `Starting task manager`. it means you are online.

## 1. Testing Pytorch Model Using Predefiend Evalution Metrics at each Round of Federeated Training

Declare a torch.nn MyTrainingPlan class to send for training on the node.

In [ ]:
import torch
import torch.nn as nn
from fedbiomed.common.training_plans import TorchTrainingPlan
from fedbiomed.common.data import DataManager
from torchvision import datasets, transforms

# Here we define the model to be used. 
# You can use any class name (here 'Net')
class MyTrainingPlan(TorchTrainingPlan):
    def __init__(self, model_args: dict = {}):
        super(MyTrainingPlan, self).__init__(model_args)
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout(0.25)
        self.dropout2 = nn.Dropout(0.5)
        self.fc1 = nn.Linear(9216, 128)
        self.fc2 = nn.Linear(128, 10)
        
        # Here we define the custom dependencies that will be needed by our custom Dataloader
        # In this case, we need the torch DataLoader classes
        # Since we will train on MNIST, we need datasets and transform from torchvision
        deps = ["from torchvision import datasets, transforms"]
        
        self.add_dependency(deps)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        
        
        output = F.log_softmax(x, dim=1)
        return output

    def training_data(self, batch_size = 48):
        # Custom torch Dataloader for MNIST data
        transform = transforms.Compose([transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))])
        dataset1 = datasets.MNIST(self.dataset_path, train=True, download=False, transform=transform)
        train_kwargs = {'batch_size': batch_size, 'shuffle': True}
        return DataManager(dataset=dataset1, **train_kwargs)
    
    def training_step(self, data, target):
        output = self.forward(data)
        loss   = torch.nn.functional.nll_loss(output, target)
        return loss


### 3.1 Declare and run the experiment
The model is trained on the **MNIST dataset** for classification. For testing, we will be using the **F1-Score**  as a metric. Testing will be performed on both **local updates and global updates**.

In [ ]:
from fedbiomed.researcher.experiment import Experiment
from fedbiomed.researcher.aggregators.fedavg import FedAverage


model_args = {}

training_args = {
    'batch_size': 48, 
    'lr': 1e-3, 
    'epochs': 1, 
    'dry_run': False,  
    'batch_maxnum': 100, # Fast pass for development : only use ( batch_maxnum * batch_size ) samples
  
}


tags =  ['#MNIST', '#dataset']
rounds = 2

exp = Experiment(tags=tags,
                 model_args=model_args,
                 model_class=MyTrainingPlan,
                 training_args=training_args,
                 round_limit=rounds,
                 aggregator=FedAverage(),
                 node_selection_strategy=None,
                tensorboard=True)

#### Declaring Testing Arguments 

- **test_ratio:** The ratio for testing partition 
- **test_metric:** The metric that is going to be used for evaluation
- **Testing on local updates:** Means that testing is going to be perform after training is performed over aggreated paramaters  
- **Testing on global updates**: Means that testing will be perform on aggregated parameters before performing the training. 


You can display all the default metrics that are supported in Fed-BioMed. They are all based on sklearn metrics

In [ ]:
from fedbiomed.common.metrics import MetricTypes
MetricTypes.get_all_metrics()

In [ ]:
exp.set_test_ratio(0.1)
exp.set_test_on_local_updates(True)
exp.set_test_on_global_updates(True)
exp.set_test_metric(MetricTypes.F1_SCORE)

Launch tensorboard

In [1]:
from fedbiomed.researcher.environ import environ
tensorboard_dir = environ['TENSORBOARD_RESULTS_DIR']

2022-04-04 15:29:46,729 fedbiomed INFO - Component environment:
2022-04-04 15:29:46,730 fedbiomed INFO - type = ComponentType.RESEARCHER


In [2]:
%load_ext tensorboard

In [3]:
tensorboard --logdir "$tensorboard_dir"

Let's start the experiment.

By default, this function doesn't stop until all the `round_limit` rounds are done for all the nodes

In [ ]:
exp.run()



## 2. Training and Testing with sklearn Perceptron model


Now we will use the testing facility on Skelearn training plan

In [ ]:
from fedbiomed.common.training_plans import SGDSkLearnModel
from fedbiomed.common.data import DataManager
import numpy as np


class SkLearnClassifierTrainingPlan(SGDSkLearnModel):
    def __init__(self, model_args):
        super(SkLearnClassifierTrainingPlan,self).__init__(model_args)
        self.add_dependency(['import torch',
                            "from sklearn.linear_model import Perceptron",
                            "from torchvision import datasets, transforms",
                           "from torch.utils.data import DataLoader"])
    
    
    def training_data(self):
        # Custom torch Dataloader for MNIST data: np.ndarray
        transform = transforms.Compose([transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))])
        dataset = datasets.MNIST(self.dataset_path, train=True, download=False, transform=transform)
        
        train_kwargs = {'batch_size': 500, 'shuffle': True}  # number of data passed to classifier
        X_train = dataset.data.numpy()
        X_train = X_train.reshape(-1, 28*28)
        Y_train = dataset.targets.numpy()
        
        return DataManager(dataset=X_train,target=Y_train)

It is also possible to define testing option in the training arguments. 

In [ ]:
model_args = { 'max_iter':1000,
              'tol': 1e-4 ,
              'model': 'Perceptron' ,
              'n_features': 28*28,
              'n_classes' : 10,
              'eta0':1e-6,
              'random_state':1234,
              'alpha':0.1 }

training_args = {
    'epochs': 5, 
}




In [ ]:
from fedbiomed.researcher.experiment import Experiment
from fedbiomed.researcher.aggregators.fedavg import FedAverage

tags =  ['#MNIST', '#dataset']
rounds = 50

# select nodes participing to this experiment
exp = Experiment(tags=tags,
                 model_args=model_args,
                 model_class=SkLearnClassifierTrainingPlan,
                 training_args=training_args,
                 round_limit=rounds,
                 aggregator=FedAverage(),
                 node_selection_strategy=None, 
                 tensorboard=True)


exp.set_test_ratio(.6)
exp.set_test_metric(MetricTypes.PRECISION, average='macro')
exp.set_test_on_global_updates(True)

In [ ]:
exp.run(increase=True)

Feel free to run other sample notebooks or try your own models :D

# 3. Testing facility using your own testing metric

If the user wants to define its own testing metric, he can do so by defining the `testing_step` method in the Training plan. 

`testing_step` is defined the same way as `training_step`:

When defining a `testing_step` method in the TrainingPlan, user has to:
- predict classes or probabilities from model
- compute a scalar or a list of scalars

Method `testing_step` can return either a scalar or a list of scalars: in Tensorboard, list of scalars will be seen as the output of several metrics


## 3.1 PyTorch Training Plan

Below we showcase an example of a TorchTrainingPlan with a `testing_step` computing 3 metrics: log likelihood loss, a cross entropy loss, and a custom accuracy metric 

In [5]:
import torch
import torch.nn as nn
from fedbiomed.common.training_plans import TorchTrainingPlan
from fedbiomed.common.data import DataManager
from torchvision import datasets, transforms

# Here we define the model to be used. 
# You can use any class name (here 'Net')
class MyTrainingPlanCM(TorchTrainingPlan):
    def __init__(self, model_args: dict = {}):
        super(MyTrainingPlanCM, self).__init__(model_args)
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout(0.25)
        self.dropout2 = nn.Dropout(0.5)
        self.fc1 = nn.Linear(9216, 128)
        self.fc2 = nn.Linear(128, 10)
        
        # Here we define the custom dependencies that will be needed by our custom Dataloader
        # In this case, we need the torch DataLoader classes
        # Since we will train on MNIST, we need datasets and transform from torchvision
        deps = ["from torchvision import datasets, transforms"]
        
        self.add_dependency(deps)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        
        
        output = F.log_softmax(x, dim=1)
        return output

    def training_data(self, batch_size = 48):
        # Custom torch Dataloader for MNIST data
        transform = transforms.Compose([transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))])
        dataset1 = datasets.MNIST(self.dataset_path, train=True, download=False, transform=transform)
        train_kwargs = {'batch_size': batch_size, 'shuffle': True}
        return DataManager(dataset=dataset1, **train_kwargs)
    
    def training_step(self, data, target):
        output = self.forward(data)
        loss   = torch.nn.functional.nll_loss(output, target)
        return loss

    def testing_step(self, data, target):        
        output = self.forward(data)
        
        #negative log likelihood loss
        loss1   = torch.nn.functional.nll_loss(output, target)
        
        #cross entropy
        loss2 = torch.nn.functional.cross_entropy(output,target)
        
        # accuracy
        _,predicted = torch.max(output.data,1)
        acc = torch.sum(predicted==target)
        loss3 = acc/len(target)

        return [loss1,loss2,loss3]

In [6]:
model_args = {}

training_args = {
    'batch_size': 48, 
    'lr': 1e-3, 
    'epochs': 1, 
    'dry_run': False,  
    'batch_maxnum': 100, # Fast pass for development : only use ( batch_maxnum * batch_size ) samples
    'test_ratio': .3,
    'test_on_local_updates': True, 
    'test_on_global_updates': True
}

In [7]:
from fedbiomed.researcher.experiment import Experiment
from fedbiomed.researcher.aggregators.fedavg import FedAverage

tags =  ['#MNIST', '#dataset']
rounds = 2

exp = Experiment(tags=tags,
                 model_args=model_args,
                 model_class=MyTrainingPlanCM,
                 training_args=training_args,
                 round_limit=rounds,
                 aggregator=FedAverage(),
                 node_selection_strategy=None, 
                tensorboard=True)

2022-04-01 17:39:18,839 fedbiomed INFO - Searching dataset with data tags: ['#MNIST', '#dataset'] for all nodes
2022-04-01 17:39:28,854 fedbiomed INFO - Node selected for training -> node_7b9015ca-a3b5-4d8d-a2bf-284b4e288f11
2022-04-01 17:39:28,894 fedbiomed DEBUG - Model file has been saved: /home/ybouilla/fedbiomed/var/experiments/Experiment_0015/my_model_3daaf88b-9d50-4ea7-99ee-ebf526011e6b.py
2022-04-01 17:39:28,924 fedbiomed DEBUG - upload (HTTP POST request) of file /home/ybouilla/fedbiomed/var/experiments/Experiment_0015/my_model_3daaf88b-9d50-4ea7-99ee-ebf526011e6b.py successful, with status code 201
2022-04-01 17:39:29,095 fedbiomed DEBUG - upload (HTTP POST request) of file /home/ybouilla/fedbiomed/var/experiments/Experiment_0015/aggregated_params_init_b5574eb6-17da-4c81-b25a-69dbe9cf9140.pt successful, with status code 201
2022-04-01 17:39:29,098 fedbiomed INFO - Removing tensorboard logs from previous experiment


In [8]:
exp.run()

2022-04-01 17:39:29,108 fedbiomed INFO - Sampled nodes in round 0 ['node_7b9015ca-a3b5-4d8d-a2bf-284b4e288f11']
2022-04-01 17:39:29,110 fedbiomed INFO - Sending request 
					 To: node_7b9015ca-a3b5-4d8d-a2bf-284b4e288f11 
					 Reqeust: : Perform training with the arguments: {'researcher_id': 'researcher_92c4776a-9b94-45da-a227-a9e8f36d7cf6', 'job_id': '3c96768f-7b90-4d62-8b8b-3a1528ebc68b', 'training_args': {'test_ratio': 0.3, 'test_on_local_updates': True, 'test_on_global_updates': True, 'test_metric': None, 'test_metric_args': {}, 'batch_size': 48, 'lr': 0.001, 'epochs': 1, 'dry_run': False, 'batch_maxnum': 100}, 'training': True, 'model_args': {}, 'command': 'train', 'model_url': 'http://localhost:8844/media/uploads/2022/04/01/my_model_3daaf88b-9d50-4ea7-99ee-ebf526011e6b.py', 'params_url': 'http://localhost:8844/media/uploads/2022/04/01/aggregated_params_init_b5574eb6-17da-4c81-b25a-69dbe9cf9140.pt', 'model_class': 'MyTrainingPlanCM', 'training_data': {'node_7b9015ca-a3b5-4d8d-a2

2022-04-01 17:40:09,816 fedbiomed INFO - TRAINING 
					 NODE_ID: node_7b9015ca-a3b5-4d8d-a2bf-284b4e288f11 
					 Epoch: 1 | Completed: 2400/42000 (6%) 
 					 Loss: 0.174411 
					 ---------
2022-04-01 17:40:10,426 fedbiomed INFO - TRAINING 
					 NODE_ID: node_7b9015ca-a3b5-4d8d-a2bf-284b4e288f11 
					 Epoch: 1 | Completed: 2880/42000 (7%) 
 					 Loss: 0.307284 
					 ---------
2022-04-01 17:40:10,935 fedbiomed INFO - TRAINING 
					 NODE_ID: node_7b9015ca-a3b5-4d8d-a2bf-284b4e288f11 
					 Epoch: 1 | Completed: 3360/42000 (8%) 
 					 Loss: 0.259756 
					 ---------
2022-04-01 17:40:11,544 fedbiomed INFO - TRAINING 
					 NODE_ID: node_7b9015ca-a3b5-4d8d-a2bf-284b4e288f11 
					 Epoch: 1 | Completed: 3840/42000 (9%) 
 					 Loss: 0.063521 
					 ---------
2022-04-01 17:40:12,167 fedbiomed INFO - TRAINING 
					 NODE_ID: node_7b9015ca-a3b5-4d8d-a2bf-284b4e288f11 
					 Epoch: 1 | Completed: 4320/42000 (10%) 
 					 Loss: 0.137890 
					 ---------
2022-04-01 17:40:21,293 fedbiomed 

2

## 3.2 Sklearn Training Plan

Below we showcase an example of a SklearnTrainingPlan with a `testing_step` computing several metrics

In [4]:
from fedbiomed.common.training_plans import SGDSkLearnModel
from fedbiomed.common.data import DataManager
import numpy as np


class SkLearnClassifierTrainingPlan(SGDSkLearnModel):
    def __init__(self, model_args):
        super(SkLearnClassifierTrainingPlan,self).__init__(model_args)
        self.add_dependency(['import torch',
                            "from sklearn.linear_model import Perceptron",
                            "from torchvision import datasets, transforms",
                           "from torch.utils.data import DataLoader",
                            "from sklearn.metrics import hinge_loss"])
    
    
    def compute_accuracy_for_specific_digit(self, data, target, digit: int):
        idx_data_equal_to_digit = target == digit
        
        predicted = self.model.predict(data[idx_data_equal_to_digit])
        well_predicted_label = np.sum(predicted == digit) / np.sum(idx_data_equal_to_digit)
        return well_predicted_label
    
    def training_data(self):
        # Custom torch Dataloader for MNIST data
        transform = transforms.Compose([transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))])
        dataset = datasets.MNIST(self.dataset_path, train=True, download=False, transform=transform)
        
        train_kwargs = {'batch_size': 500, 'shuffle': True}  # number of data passed to classifier
        X_train = dataset.data.numpy()
        X_train = X_train.reshape(-1, 28*28)
        Y_train = dataset.targets.numpy()
        
        return DataManager(dataset=X_train, target=Y_train)
    
    def testing_step(self, data, target):
        #test_data = data.reshape(-1, 28 * 28)
        # hinge loss
        distance_from_hyperplan = self.model.decision_function(data)
        loss = hinge_loss(target, distance_from_hyperplan)
        
        # get the accuracy only on images representing digit 1
        well_predicted_label_1 = self.compute_accuracy_for_specific_digit(data, target, 1)
        
        return [loss, well_predicted_label_1]

In [5]:
model_args = { 'max_iter':1000,
              'tol': 1e-4 ,
              'model': 'Perceptron' ,
              'n_features': 28*28,
              'n_classes' : 10,
              'eta0':1e-6,
              'random_state':1234,
              'alpha':0.1 }

training_args = {
    'epochs': 5, 
}


In [6]:
from fedbiomed.researcher.experiment import Experiment
from fedbiomed.researcher.aggregators.fedavg import FedAverage

tags =  ['#MNIST', '#dataset']
rounds = 200

# select nodes participing to this experiment
exp = Experiment(tags=tags,
                 model_args=model_args,
                 model_class=SkLearnClassifierTrainingPlan,
                 training_args=training_args,
                 round_limit=rounds,
                 aggregator=FedAverage(),
                 node_selection_strategy=None, 
                 tensorboard=True)


exp.set_test_ratio(.2)
#exp.set_test_metric(MetricTypes.PRECISION, average='macro')
exp.set_test_on_global_updates(True)
exp.set_test_on_local_updates(True)

2022-04-04 15:29:56,219 fedbiomed INFO - Messaging researcher_92c4776a-9b94-45da-a227-a9e8f36d7cf6 successfully connected to the message broker, object = <fedbiomed.common.messaging.Messaging object at 0x7fd2c9a5e430>
2022-04-04 15:29:56,250 fedbiomed INFO - Searching dataset with data tags: ['#MNIST', '#dataset'] for all nodes
2022-04-04 15:30:06,263 fedbiomed INFO - Node selected for training -> node_7b9015ca-a3b5-4d8d-a2bf-284b4e288f11
2022-04-04 15:30:06,270 fedbiomed DEBUG - Model file has been saved: /home/ybouilla/fedbiomed/var/experiments/Experiment_0036/my_model_18329669-84f9-4fcf-a924-2125aadff6ff.py
2022-04-04 15:30:06,294 fedbiomed DEBUG - upload (HTTP POST request) of file /home/ybouilla/fedbiomed/var/experiments/Experiment_0036/my_model_18329669-84f9-4fcf-a924-2125aadff6ff.py successful, with status code 201
2022-04-04 15:30:06,313 fedbiomed DEBUG - upload (HTTP POST request) of file /home/ybouilla/fedbiomed/var/experiments/Experiment_0036/aggregated_params_init_8b1c2878-

True

In [ ]:
exp.run(increase=True)

2022-04-04 15:30:06,327 fedbiomed INFO - Sampled nodes in round 0 ['node_7b9015ca-a3b5-4d8d-a2bf-284b4e288f11']
2022-04-04 15:30:06,329 fedbiomed INFO - Sending request 
					 To: node_7b9015ca-a3b5-4d8d-a2bf-284b4e288f11 
					 Reqeust: : Perform training with the arguments: {'researcher_id': 'researcher_92c4776a-9b94-45da-a227-a9e8f36d7cf6', 'job_id': 'd9ad5af4-4ee9-4c77-9885-adcd5439f1a6', 'training_args': {'test_ratio': 0.2, 'test_on_local_updates': True, 'test_on_global_updates': True, 'test_metric': None, 'test_metric_args': {}, 'epochs': 5}, 'training': True, 'model_args': {'max_iter': 1000, 'tol': 0.0001, 'model': 'Perceptron', 'n_features': 784, 'n_classes': 10, 'eta0': 1e-06, 'random_state': 1234, 'alpha': 0.1, 'verbose': 1}, 'command': 'train', 'model_url': 'http://localhost:8844/media/uploads/2022/04/04/my_model_18329669-84f9-4fcf-a924-2125aadff6ff.py', 'params_url': 'http://localhost:8844/media/uploads/2022/04/04/aggregated_params_init_8b1c2878-ffeb-4132-be14-ab3c86b4a663.

2022-04-04 15:30:17,187 fedbiomed INFO - TRAINING 
					 NODE_ID: node_7b9015ca-a3b5-4d8d-a2bf-284b4e288f11 
					 Epoch: 0 | Completed: 48000/48000 (100%) 
 					 Loss perceptron: 0.100233 
					 ---------
2022-04-04 15:30:17,838 fedbiomed INFO - WARNING
					 NODE node_7b9015ca-a3b5-4d8d-a2bf-284b4e288f11
					 MESSAGE: Loss plot displayed on Tensorboard may be inaccurate (due to some plain SGD scikit learn limitations)
-----------------------------------------------------------------
2022-04-04 15:30:17,838 fedbiomed INFO - TRAINING 
					 NODE_ID: node_7b9015ca-a3b5-4d8d-a2bf-284b4e288f11 
					 Epoch: 1 | Completed: 48000/48000 (100%) 
 					 Loss perceptron: 0.097910 
					 ---------
2022-04-04 15:30:18,511 fedbiomed INFO - WARNING
					 NODE node_7b9015ca-a3b5-4d8d-a2bf-284b4e288f11
					 MESSAGE: Loss plot displayed on Tensorboard may be inaccurate (due to some plain SGD scikit learn limitations)
-----------------------------------------------------------------
2022-04-04 15:30

2022-04-04 15:30:29,844 fedbiomed INFO - TRAINING 
					 NODE_ID: node_7b9015ca-a3b5-4d8d-a2bf-284b4e288f11 
					 Epoch: 4 | Completed: 48000/48000 (100%) 
 					 Loss perceptron: 0.103682 
					 ---------
2022-04-04 15:30:29,912 fedbiomed INFO - TESTING ON LOCAL UPDATES 
					 NODE_ID: node_7b9015ca-a3b5-4d8d-a2bf-284b4e288f11 
					 Completed: 12000/12000 (100%) 
 					 Custom_1: 0.751318 
					 Custom_2: 0.966368 
					 ---------
2022-04-04 15:30:29,944 fedbiomed INFO - INFO
					 NODE node_7b9015ca-a3b5-4d8d-a2bf-284b4e288f11
					 MESSAGE: results uploaded successfully 
-----------------------------------------------------------------
2022-04-04 15:30:36,479 fedbiomed INFO - Downloading model params after training on node_7b9015ca-a3b5-4d8d-a2bf-284b4e288f11 - from http://localhost:8844/media/uploads/2022/04/04/node_params_2053b9f8-ac2b-4933-93b0-dea5df15482e.pt
2022-04-04 15:30:36,490 fedbiomed DEBUG - upload (HTTP GET request) of file node_params_a36a5a8f-4874-41ef-8900-0d43116

2022-04-04 15:30:46,590 fedbiomed DEBUG - researcher_92c4776a-9b94-45da-a227-a9e8f36d7cf6
2022-04-04 15:30:46,670 fedbiomed INFO - INFO
					 NODE node_7b9015ca-a3b5-4d8d-a2bf-284b4e288f11
					 MESSAGE: training with arguments {'history_monitor': <fedbiomed.node.history_monitor.HistoryMonitor object at 0x7f3042586a90>, 'node_args': {'gpu': False, 'gpu_num': None, 'gpu_only': False}, 'epochs': 5}
-----------------------------------------------------------------
2022-04-04 15:30:46,720 fedbiomed INFO - TESTING ON GLOBAL UPDATES 
					 NODE_ID: node_7b9015ca-a3b5-4d8d-a2bf-284b4e288f11 
					 Completed: 12000/12000 (100%) 
 					 Custom_1: 0.784290 
					 Custom_2: 0.985660 
					 ---------
2022-04-04 15:30:47,393 fedbiomed INFO - WARNING
					 NODE node_7b9015ca-a3b5-4d8d-a2bf-284b4e288f11
					 MESSAGE: Loss plot displayed on Tensorboard may be inaccurate (due to some plain SGD scikit learn limitations)
-----------------------------------------------------------------
2022-04-04 15:30:

2022-04-04 15:30:58,787 fedbiomed INFO - TRAINING 
					 NODE_ID: node_7b9015ca-a3b5-4d8d-a2bf-284b4e288f11 
					 Epoch: 2 | Completed: 48000/48000 (100%) 
 					 Loss perceptron: 0.085396 
					 ---------
2022-04-04 15:30:59,490 fedbiomed INFO - WARNING
					 NODE node_7b9015ca-a3b5-4d8d-a2bf-284b4e288f11
					 MESSAGE: Loss plot displayed on Tensorboard may be inaccurate (due to some plain SGD scikit learn limitations)
-----------------------------------------------------------------
2022-04-04 15:30:59,491 fedbiomed INFO - TRAINING 
					 NODE_ID: node_7b9015ca-a3b5-4d8d-a2bf-284b4e288f11 
					 Epoch: 3 | Completed: 48000/48000 (100%) 
 					 Loss perceptron: 0.085125 
					 ---------
2022-04-04 15:31:00,159 fedbiomed INFO - WARNING
					 NODE node_7b9015ca-a3b5-4d8d-a2bf-284b4e288f11
					 MESSAGE: Loss plot displayed on Tensorboard may be inaccurate (due to some plain SGD scikit learn limitations)
-----------------------------------------------------------------
2022-04-04 15:31

...

2022-04-04 15:45:59,560 fedbiomed INFO - TRAINING 
					 NODE_ID: node_7b9015ca-a3b5-4d8d-a2bf-284b4e288f11 
					 Epoch: 2 | Completed: 48000/48000 (100%) 
 					 Loss perceptron: 0.113651 
					 ---------
2022-04-04 15:46:00,252 fedbiomed INFO - WARNING
					 NODE node_7b9015ca-a3b5-4d8d-a2bf-284b4e288f11
					 MESSAGE: Loss plot displayed on Tensorboard may be inaccurate (due to some plain SGD scikit learn limitations)
-----------------------------------------------------------------
2022-04-04 15:46:00,253 fedbiomed INFO - TRAINING 
					 NODE_ID: node_7b9015ca-a3b5-4d8d-a2bf-284b4e288f11 
					 Epoch: 3 | Completed: 48000/48000 (100%) 
 					 Loss perceptron: 0.112715 
					 ---------
2022-04-04 15:46:00,944 fedbiomed INFO - WARNING
					 NODE node_7b9015ca-a3b5-4d8d-a2bf-284b4e288f11
					 MESSAGE: Loss plot displayed on Tensorboard may be inaccurate (due to some plain SGD scikit learn limitations)
-----------------------------------------------------------------
2022-04-04 15:46

2022-04-04 15:46:17,481 fedbiomed INFO - Nodes that successfully reply in round 94 ['node_7b9015ca-a3b5-4d8d-a2bf-284b4e288f11']
2022-04-04 15:46:17,508 fedbiomed DEBUG - upload (HTTP POST request) of file /home/ybouilla/fedbiomed/var/experiments/Experiment_0036/aggregated_params_0b769b13-4f49-476c-a70a-1bb769e7ea04.pt successful, with status code 201
2022-04-04 15:46:17,508 fedbiomed INFO - Saved aggregated params for round 94 in /home/ybouilla/fedbiomed/var/experiments/Experiment_0036/aggregated_params_0b769b13-4f49-476c-a70a-1bb769e7ea04.pt
2022-04-04 15:46:17,509 fedbiomed INFO - Sampled nodes in round 95 ['node_7b9015ca-a3b5-4d8d-a2bf-284b4e288f11']
2022-04-04 15:46:17,509 fedbiomed INFO - Sending request 
					 To: node_7b9015ca-a3b5-4d8d-a2bf-284b4e288f11 
					 Reqeust: : Perform training with the arguments: {'researcher_id': 'researcher_92c4776a-9b94-45da-a227-a9e8f36d7cf6', 'job_id': 'd9ad5af4-4ee9-4c77-9885-adcd5439f1a6', 'training_args': {'test_ratio': 0.2, 'test_on_local_u

2022-04-04 15:46:27,699 fedbiomed INFO - TESTING ON GLOBAL UPDATES 
					 NODE_ID: node_7b9015ca-a3b5-4d8d-a2bf-284b4e288f11 
					 Completed: 12000/12000 (100%) 
 					 Custom_1: 0.682246 
					 Custom_2: 0.911721 
					 ---------
2022-04-04 15:46:28,391 fedbiomed INFO - WARNING
					 NODE node_7b9015ca-a3b5-4d8d-a2bf-284b4e288f11
					 MESSAGE: Loss plot displayed on Tensorboard may be inaccurate (due to some plain SGD scikit learn limitations)
-----------------------------------------------------------------
2022-04-04 15:46:28,392 fedbiomed INFO - TRAINING 
					 NODE_ID: node_7b9015ca-a3b5-4d8d-a2bf-284b4e288f11 
					 Epoch: 0 | Completed: 48000/48000 (100%) 
 					 Loss perceptron: 0.082008 
					 ---------
2022-04-04 15:46:29,033 fedbiomed INFO - WARNING
					 NODE node_7b9015ca-a3b5-4d8d-a2bf-284b4e288f11
					 MESSAGE: Loss plot displayed on Tensorboard may be inaccurate (due to some plain SGD scikit learn limitations)
---------------------------------------------------------

2022-04-04 15:46:40,458 fedbiomed INFO - TRAINING 
					 NODE_ID: node_7b9015ca-a3b5-4d8d-a2bf-284b4e288f11 
					 Epoch: 3 | Completed: 48000/48000 (100%) 
 					 Loss perceptron: 0.102014 
					 ---------
2022-04-04 15:46:41,168 fedbiomed INFO - WARNING
					 NODE node_7b9015ca-a3b5-4d8d-a2bf-284b4e288f11
					 MESSAGE: Loss plot displayed on Tensorboard may be inaccurate (due to some plain SGD scikit learn limitations)
-----------------------------------------------------------------
2022-04-04 15:46:41,170 fedbiomed INFO - TRAINING 
					 NODE_ID: node_7b9015ca-a3b5-4d8d-a2bf-284b4e288f11 
					 Epoch: 4 | Completed: 48000/48000 (100%) 
 					 Loss perceptron: 0.101728 
					 ---------
2022-04-04 15:46:41,220 fedbiomed INFO - TESTING ON LOCAL UPDATES 
					 NODE_ID: node_7b9015ca-a3b5-4d8d-a2bf-284b4e288f11 
					 Completed: 12000/12000 (100%) 
 					 Custom_1: 0.737217 
					 Custom_2: 0.906810 
					 ---------
2022-04-04 15:46:41,251 fedbiomed INFO - INFO
					 NODE node_7b9015ca

2022-04-04 15:46:57,776 fedbiomed INFO - Sampled nodes in round 99 ['node_7b9015ca-a3b5-4d8d-a2bf-284b4e288f11']
2022-04-04 15:46:57,776 fedbiomed INFO - Sending request 
					 To: node_7b9015ca-a3b5-4d8d-a2bf-284b4e288f11 
					 Reqeust: : Perform training with the arguments: {'researcher_id': 'researcher_92c4776a-9b94-45da-a227-a9e8f36d7cf6', 'job_id': 'd9ad5af4-4ee9-4c77-9885-adcd5439f1a6', 'training_args': {'test_ratio': 0.2, 'test_on_local_updates': True, 'test_on_global_updates': True, 'test_metric': None, 'test_metric_args': {}, 'epochs': 5}, 'training': True, 'model_args': {'max_iter': 1000, 'tol': 0.0001, 'model': 'Perceptron', 'n_features': 784, 'n_classes': 10, 'eta0': 1e-06, 'random_state': 1234, 'alpha': 0.1, 'verbose': 1}, 'command': 'train', 'model_url': 'http://localhost:8844/media/uploads/2022/04/04/my_model_18329669-84f9-4fcf-a924-2125aadff6ff.py', 'params_url': 'http://localhost:8844/media/uploads/2022/04/04/aggregated_params_4d99c8f4-93e0-4fab-b057-5704b92c229b.pt',